# First Task

#### 1. Install BeautifulSoup4 library

We need to install beautifulsoup4 library in the first place for web scrapping in this task.

In [1]:
conda install -c anaconda beautifulsoup4

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Siahaan\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.1       |           py37_0         153 KB  anaconda
    certifi-2019.11.28         |           py37_0         157 KB  anaconda
    conda-4.8.0                |           py37_1         3.1 MB  anaconda
    ------------------------------------------------------------
                                           Total:         3.4 MB

The following packages will be UPDATED:

  beautifulsoup4     pkgs/main::beautifulsoup4-4.7.1-py37_1 --> anaconda::beautifulsoup4-4.8.1-py37_0
  openssl            conda-forge::openssl-1.1.1d-hfa6e2cd_0 --> anaconda::openssl-1.1.1-he774522_0

The following packages will be SUPERSEDED by a higher-priority channel:


#### 2. Import all needed libraries
All necessary library imported in this project.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

#### 3. Web Scrapping the wikipedia page containing list of neighborhoods in Toronto
The following lines of code are for web scrapping the wikipedia page that contains list of neighborhoods in the city of Toronto.

In [2]:
# URL of the wikipedia page
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

# Create beautiful soup object
soup = BeautifulSoup(source, 'lxml')

# Find table that contains the neighborhoods data in the page
table = soup.find('table').tbody

# Get all columns name
columns = [item.text.replace("\n", "") for item in table.find_all("th")]

# Get all neighborhood data
neighborhoods_data = []
for i,row in enumerate(table.find_all('tr')):
    neighborhoods_data.append([])
    for j, item in enumerate(row.find_all('td')):
        neighborhoods_data[i].append(item.text.replace("\n", ""))

neighborhoods_data = neighborhoods_data[1:]
neighborhoods_data = list(map(list, zip(*neighborhoods_data)))

# Create Dataframe
tor_neigh_df = pd.DataFrame({columns[i]: neighborhoods_data[i] for i in range(3)})
tor_neigh_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 4. Rename PostalCode Column
We want to rename one column in our dataframe.

In [3]:
tor_neigh_df.rename(columns={"Postcode": "PostalCode"}, inplace=True)
tor_neigh_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 5. Drop Not Assigned value in Borough Column
All rows with not assigned borough value will be dropped.

In [4]:
tor_neigh_df = tor_neigh_df[tor_neigh_df["Borough"]!="Not assigned"]
tor_neigh_df.reset_index(drop=True, inplace=True)
tor_neigh_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### 6. Cell with Not Assigned neighborhood will be filled with the same value as the borough 

In [5]:
# Function to check neighborhood value and process it if it is still not assigned
def replace_neighborhood_na(row):
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
    return row

In [6]:
tor_neigh_df = tor_neigh_df.apply(replace_neighborhood_na, axis=1)
tor_neigh_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### 7. Combine rows with same PostalCode

In [7]:
neigh_df = tor_neigh_df.groupby(by="PostalCode").agg({"Borough": 'first', "Neighborhood": ", ".join})
neigh_df.reset_index(inplace=True)
neigh_df.tail()

,PostalCode,Borough,Neighborhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


shape of the Dataframe

In [8]:
neigh_df.shape

(103, 3)

# Second Task

#### 1. Install geocoder library
This library is needed for getting longitude and latitude of each neighborhood.

In [24]:
conda install -c conda-forge geocoder

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Siahaan\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    conda-4.8.0                |           py37_1         3.0 MB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2

The following packages will be UPDATED:

  ca-certificates    an

#### 2. Get all longitude and latitude of each neighborhood
##### a. Using Geocoder

In [ ]:
import geocoder # import geocoder

latitudes = []
longitudes = []

for index, row in neigh_df.iterrows():
    print("Getting latitude and longitude of {}, {}, {}".format(row["PostalCode"], row["Borough"], row["Neighborhood"]))
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, {}, {}'.format(row["PostalCode"], row["Borough"], row["Neighborhood"]))
      lat_lng_coords = g.latlng

    latitudes.append(lat_lng_coords[0])
    longitudes.append(lat_lng_coords[1])
    print("                                                      [DONE]")

Getting latitude and longitude of M1B, Scarborough, Rouge, Malvern


##### b. Using available longitude and latitude data in csv format

In [12]:
ll_df = pd.read_csv("Geospatial_Coordinates.csv")
ll_df.rename(columns={"Postal Code": "PostalCode"}, inplace=True) #rename postal code column
ll_df.set_index("PostalCode", inplace=True)
ll_df.head()

,Latitude,Longitude
PostalCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


#### 3. Merge these two dataframes
We want to assign latitude and longitude of every neighborhood.

In [18]:
neigh_df.set_index("PostalCode", inplace=True)
neigh_df.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [19]:
neigh_df = neigh_df.join(ll_df)
neigh_df.head()

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
neigh_df.reset_index(inplace=True)
neigh_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
